In [12]:
import numpy as np
from matplotlib import pyplot as plt
from dataclasses import dataclass
from amplitude_robust_spectroscopy.simulation.args import QubitConfig
from amplitude_robust_spectroscopy import AmplitudeSweepSpectroscopy, OptionsSpectroscopy2d
from amplitude_robust_spectroscopy.simulation.args import *

ImportError: cannot import name 'AmplitudeSweepSpectroscopy' from 'amplitude_robust_spectroscopy' (/Users/asafsolonnikov/Developer/amplitude-robust-spectroscopy/amplitude_robust_spectroscopy/__init__.py)

In [10]:
options = OptionsSpectroscopy2d(plot=True)
qubit_config = QubitConfig()

detunings = np.linspace(-5, 5, 101)*2*np.pi*MHz
amplitudes = np.linspace(0, 10, 5)*2*np.pi*MHz
spectroscopy = AmplitudeSweepSpectroscopy(amplitudes, detunings, qubit_config,
                            options=Options(plot=True))

results = spectroscopy.run()

NameError: name 'OptionsSpectroscopy2d' is not defined